In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 21

In [4]:
name = list[k].getName()
name

'RP60'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=1)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.8f,%.8f] " % ( resultsMC[4], resultsMC[5])


b = "[%.8f,%.8f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.8f,%.8f] " % ( resultSubset[4], resultSubset[5])


In [8]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.03359519,0.04995572] ', None],
 [None, None, None, '[0.03359519,0.04995572] ', None],
 [None, None, None, '[0.03359519,0.04995572] ', None],
 [None, None, None, '[0.03359519,0.04995572] ', None],
 [None, None, None, '[0.03359519,0.04995572] ', None],
 [None, None, None, '[0.03359519,0.04995572] ', None]]

In [9]:
l = np.array([[1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1],[1, 1, 1, a, 1, 1]])

In [10]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité exacte de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  result[i][2]

results[0][3] = "-----------------"
results[1][3] = "-----------------"
results[2][3] = a

results[3][3] = c
 
results[4][3] = b


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2,x3,x4,x5]->[var g1 := x1 - x5;var g2 := x2 - x5 / 2;var g3 := x3 - x5 / 2;var g4 := x4 - x5 / 2;var g5 := x2 - x5;var g6 := x3 - x5;var g7 := x4 - x5;var g8 := min(g5, g6);var g9 := max(g7, g8);var g10 := min(g2, g3, g4);var g11 := max(g10, g9);gsys := min(g1, g11)]


,Probabilité exacte de RP60,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,4.6272494e-02,4.1775458e-02,4.497037e-03,-----------------,2298,1.0123964763710995
FORM,4.6272494e-02,4.4839762e-02,1.432732e-03,-----------------,100,1.5091579512962725
SORM,4.6272494e-02,4.483973e-02,1.4327645e-03,"[0.03359519,0.04995572]",161,1.509148096067478
FORM-IS,4.6272494e-02,6.4612165e-02,1.8339671e-02,"[0.04499741,0.04878259]",102,0.4019313665821361
SUBSET,4.6272494e-02,4.689e-02,6.1750645e-04,"[0.05354775,0.07567658]",20000,1.874681421876585


In [11]:
df.to_csv('problem60.csv')